# Create pyCIAM Storm Costs Lookup Table

Calculating the storm costs in a CIAM model involves a numerical integration over both elevation and the quantiles of storm surge at each segment-ADM1 location. This is too computationally intensive to run for all seg-ADMs for each year for all SLR trajectories, especially when using pyCIAM to run a Monte Carlo analysis across tens of thousands of SLR trajectories. Instead, we build a lookup table indexed by seg-ADM, LSLR, adaptation type (retreat vs. protect), cost type (mortality vs. capital loss), and `rhdiff` (the difference between the retreat/protect height and lslr). This is similar to how it is treated in the original CIAM model except that:

1. We use a lookup table rather than a parameterized exponential function of `rhdiff` and `lslr`
2. We account for elevational heterogeneity in population and capital when evaluating our costs in retreat scenarios. The original CIAM included `lslr` in their exponential function only for the protect adaptation type, while for `noAdaptation` and `retreat`, the function was only of `rhdiff`.

In [1]:
%load_ext autoreload
%autoreload 2

## Setup

In [2]:
import distributed as dd
import pandas as pd
from dask_gateway import Gateway
from shared import (
    PATH_PARAMS,
    PATH_SLIIDERS_ECON,
    PATH_SLIIDERS_ECON_SEG,
    PATH_SLIIDERS_SLR_QUANTILES,
    PATH_SURGE_LOOKUP,
    PATH_SURGE_LOOKUP_SEG,
    upload_pkg,
)

from pyCIAM.surge import damage_funcs
from pyCIAM.surge.lookup import create_surge_lookup

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [3]:
N_WORKERS = 700
SEG_CHUNKSIZE = 4

PARAMS = pd.read_json(PATH_PARAMS)["values"]

In [4]:
DMF_I = getattr(damage_funcs, PARAMS.dmf + "_i")
DDF_I = getattr(damage_funcs, PARAMS.ddf + "_i")

In [5]:
gateway = Gateway()
cluster = gateway.new_cluster(
    idle_timeout=3600,
    profile="micro",
    env_items={
        "DASK_DISTRIBUTED__WORKER__MEMORY__TARGET": "0.95",
        "DASK_DISTRIBUTED__WORKER__MEMORY__SPILL": "0.95",
        "DASK_DISTRIBUTED__WORKER__MEMORY__PAUSE": "0.95",
        "DASK_DISTRIBUTED__WORKER__MEMORY__TERMINATE": "0.99",
    },
)
client = cluster.get_client()
cluster.scale(N_WORKERS)

client.upload_file("shared.py")
upload_pkg(client, "../pyCIAM")

cluster

## Run surge damage calculations for each combo

In [7]:
client.wait_for_workers(N_WORKERS * 0.75)

In [21]:
futs = create_surge_lookup(
    PATH_SLIIDERS_ECON,
    PATH_SLIIDERS_SLR_QUANTILES,
    PATH_SURGE_LOOKUP,
    "seg_adm",
    PARAMS.at_start,
    PARAMS.n_interp_pts_lslr,
    PARAMS.n_interp_pts_rhdiff,
    DDF_I,
    DMF_I,
    client=client,
    client_kwargs={"batch_size": N_WORKERS},
    force_overwrite=False,
    dmf_kwargs={"floodmortality": PARAMS.floodmortality},
    seg_chunksize=4,
)

In [23]:
futs_seg = create_surge_lookup(
    PATH_SLIIDERS_ECON_SEG,
    PATH_SLIIDERS_SLR_QUANTILES,
    PATH_SURGE_LOOKUP_SEG,
    "seg",
    PARAMS.at_start,
    PARAMS.n_interp_pts_lslr,
    PARAMS.n_interp_pts_rhdiff,
    DDF_I,
    DMF_I,
    client=client,
    client_kwargs={"batch_size": N_WORKERS},
    force_overwrite=True,
    dmf_kwargs={"floodmortality": PARAMS.floodmortality},
    seg_chunksize=4,
)

## Close

In [32]:
# ensure completion and close cluster
dd.wait(futs + futs_seg)
cluster.close(), client.close()

(None, None)